In [1]:
# Feature Engineering for Transaction Volume Forecasting
# Add features to improve model performance

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import storage

# Configuration
PROJECT_ID = "transaction-forecast-mlops"
BUCKET_NAME = "transaction-forecast-data"

print("="*60)
print("FEATURE ENGINEERING")
print("="*60)

# Load clean data
print("\nLoading clean transaction data...")
df = pd.read_csv(f'gs://{BUCKET_NAME}/processed_data/daily_volumes_clean.csv')
df['date'] = pd.to_datetime(df['date'])

print(f"✓ Loaded {len(df)} days")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"\nCurrent columns: {df.columns.tolist()}")
print(f"\nStarting feature engineering...")

FEATURE ENGINEERING

Loading clean transaction data...
✓ Loaded 610 days
Date range: 2016-09-04 00:00:00 to 2018-08-22 00:00:00

Current columns: ['date', 'transaction_volume']

Starting feature engineering...


In [2]:
# 1. TEMPORAL FEATURES
print("\n[1/5] Adding temporal features...")

df['day_of_week'] = df['date'].dt.dayofweek  # 0=Monday, 6=Sunday
df['day_name'] = df['date'].dt.day_name()
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['day_of_month'] = df['date'].dt.day
df['week_of_year'] = df['date'].dt.isocalendar().week
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
df['is_month_end'] = df['date'].dt.is_month_end.astype(int)

print(f"✓ Added {9} temporal features")
print(f"  - day_of_week, day_name, month, quarter")
print(f"  - is_weekend, is_month_start, is_month_end")
print(f"  - day_of_month, week_of_year")


[1/5] Adding temporal features...
✓ Added 9 temporal features
  - day_of_week, day_name, month, quarter
  - is_weekend, is_month_start, is_month_end
  - day_of_month, week_of_year


In [3]:
# 2. LAG FEATURES (historical values)
print("\n[2/5] Adding lag features...")

# Shift transaction volumes back in time
df['lag_1'] = df['transaction_volume'].shift(1)   # Yesterday
df['lag_7'] = df['transaction_volume'].shift(7)   # Last week same day
df['lag_14'] = df['transaction_volume'].shift(14) # Two weeks ago
df['lag_30'] = df['transaction_volume'].shift(30) # Last month

print(f"✓ Added 4 lag features")
print(f"  - lag_1 (yesterday)")
print(f"  - lag_7 (last week)")
print(f"  - lag_14 (two weeks ago)")
print(f"  - lag_30 (last month)")

print(f"\nSample with lag features:")
print(df[['date', 'transaction_volume', 'lag_1', 'lag_7']].head(10))
print(f"\nNote: First rows have NaN for lags (no historical data yet)")


[2/5] Adding lag features...
✓ Added 4 lag features
  - lag_1 (yesterday)
  - lag_7 (last week)
  - lag_14 (two weeks ago)
  - lag_30 (last month)

Sample with lag features:
        date  transaction_volume  lag_1  lag_7
0 2016-09-04                   1    NaN    NaN
1 2016-09-05                   1    1.0    NaN
2 2016-09-13                   1    1.0    NaN
3 2016-09-15                   1    1.0    NaN
4 2016-10-02                   1    1.0    NaN
5 2016-10-03                   8    1.0    NaN
6 2016-10-04                  63    8.0    NaN
7 2016-10-05                  47   63.0    1.0
8 2016-10-06                  51   47.0    1.0
9 2016-10-07                  46   51.0    1.0

Note: First rows have NaN for lags (no historical data yet)


In [4]:
# 3. ROLLING STATISTICS (moving averages and trends)
print("\n[3/5] Adding rolling statistics...")

df['rolling_mean_7'] = df['transaction_volume'].rolling(window=7).mean()
df['rolling_mean_14'] = df['transaction_volume'].rolling(window=14).mean()
df['rolling_mean_30'] = df['transaction_volume'].rolling(window=30).mean()
df['rolling_std_7'] = df['transaction_volume'].rolling(window=7).std()
df['rolling_std_30'] = df['transaction_volume'].rolling(window=30).std()
df['rolling_min_7'] = df['transaction_volume'].rolling(window=7).min()
df['rolling_max_7'] = df['transaction_volume'].rolling(window=7).max()

print(f"✓ Added 7 rolling statistics features")

# 4. BRAZILIAN HOLIDAYS
print("\n[4/5] Adding Brazilian holidays...")

# Major Brazilian holidays that affect e-commerce
brazilian_holidays = [
    '2016-01-01', '2016-04-21', '2016-05-01', '2016-09-07', '2016-10-12', '2016-11-02', '2016-11-15', '2016-11-25', '2016-12-25',
    '2017-01-01', '2017-04-21', '2017-05-01', '2017-09-07', '2017-10-12', '2017-11-02', '2017-11-15', '2017-11-24', '2017-12-25',
    '2018-01-01', '2018-04-21', '2018-05-01', '2018-09-07', '2018-10-12', '2018-11-02', '2018-11-15', '2018-11-23', '2018-12-25'
]

df['is_holiday'] = df['date'].astype(str).isin(brazilian_holidays).astype(int)

# Days before/after holidays
df['days_to_holiday'] = 0
df['days_from_holiday'] = 0

for holiday in pd.to_datetime(brazilian_holidays):
    days_diff = (df['date'] - holiday).dt.days
    df.loc[(days_diff < 0) & (days_diff > -7), 'days_to_holiday'] = abs(days_diff)
    df.loc[(days_diff > 0) & (days_diff < 7), 'days_from_holiday'] = days_diff

print(f"✓ Added 3 holiday features")
print(f"  - is_holiday, days_to_holiday, days_from_holiday")

# 5. TREND FEATURES
print("\n[5/5] Adding trend features...")

df['days_since_start'] = (df['date'] - df['date'].min()).dt.days
df['transaction_growth'] = df['transaction_volume'].pct_change()
df['momentum_7'] = df['transaction_volume'].diff(7)  # Change from last week

print(f"✓ Added 3 trend features")
print(f"  - days_since_start, transaction_growth, momentum_7")

print(f"\n{'='*60}")
print(f"FEATURE ENGINEERING COMPLETE")
print(f"{'='*60}")
print(f"Total features: {len(df.columns)}")
print(f"Original: 2 columns → Enriched: {len(df.columns)} columns")


[3/5] Adding rolling statistics...
✓ Added 7 rolling statistics features

[4/5] Adding Brazilian holidays...
✓ Added 3 holiday features
  - is_holiday, days_to_holiday, days_from_holiday

[5/5] Adding trend features...
✓ Added 3 trend features
  - days_since_start, transaction_growth, momentum_7

FEATURE ENGINEERING COMPLETE
Total features: 28
Original: 2 columns → Enriched: 28 columns


In [6]:
print("Missing values check:")
missing = df.isnull().sum()
print(missing[missing > 0].sort_values(ascending=False))

Missing values check:
lag_30                30
rolling_mean_30       29
rolling_std_30        29
lag_14                14
rolling_mean_14       13
lag_7                  7
momentum_7             7
rolling_mean_7         6
rolling_std_7          6
rolling_min_7          6
rolling_max_7          6
lag_1                  1
transaction_growth     1
dtype: int64


In [7]:
# Check for missing values
print("\nMissing values check:")
missing = df.isnull().sum()
print(missing[missing > 0].sort_values(ascending=False))

print(f"\nRows with missing values: {df.isnull().any(axis=1).sum()}")
print(f"Filling missing values with forward fill method...")

df_filled = df.ffill().fillna(0)  # Forward fill, then zero

print(f"✓ Missing values handled")
print(f"\nFinal dataset shape: {df_filled.shape}")
print(f"Columns: {df_filled.columns.tolist()}")

# Save enriched dataset
df_filled.to_csv(f'gs://{BUCKET_NAME}/processed_data/daily_volumes_enriched.csv', index=False)
print(f"\n✓ Saved enriched dataset to gs://{BUCKET_NAME}/processed_data/daily_volumes_enriched.csv")

print("\n" + "="*60)
print("READY FOR ADVANCED MODELING!")
print("="*60)


Missing values check:
lag_30                30
rolling_mean_30       29
rolling_std_30        29
lag_14                14
rolling_mean_14       13
lag_7                  7
momentum_7             7
rolling_mean_7         6
rolling_std_7          6
rolling_min_7          6
rolling_max_7          6
lag_1                  1
transaction_growth     1
dtype: int64

Rows with missing values: 30
Filling missing values with forward fill method...
✓ Missing values handled

Final dataset shape: (610, 28)
Columns: ['date', 'transaction_volume', 'day_of_week', 'day_name', 'month', 'quarter', 'day_of_month', 'week_of_year', 'is_weekend', 'is_month_start', 'is_month_end', 'lag_1', 'lag_7', 'lag_14', 'lag_30', 'rolling_mean_7', 'rolling_mean_14', 'rolling_mean_30', 'rolling_std_7', 'rolling_std_30', 'rolling_min_7', 'rolling_max_7', 'is_holiday', 'days_to_holiday', 'days_from_holiday', 'days_since_start', 'transaction_growth', 'momentum_7']

✓ Saved enriched dataset to gs://transaction-forecast-data/p